In [13]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [14]:
df=pd.read_csv("pulsar_stars.csv")
df.columns

Index([' Mean of the integrated profile',
       ' Standard deviation of the integrated profile',
       ' Excess kurtosis of the integrated profile',
       ' Skewness of the integrated profile', ' Mean of the DM-SNR curve',
       ' Standard deviation of the DM-SNR curve',
       ' Excess kurtosis of the DM-SNR curve', ' Skewness of the DM-SNR curve',
       'target_class'],
      dtype='object')

In [15]:
from sklearn.preprocessing import StandardScaler

X=df.drop(["target_class"],axis=1)
Y=df["target_class"]
scaler = StandardScaler()
data_scaled=scaler.fit_transform(X)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_scaled, Y, test_size=0.33,stratify=Y, random_state=42)

In [5]:
stopping=keras.callbacks.EarlyStopping(monitor = 'val_loss',
                          
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True) #This is to efficently stop training. eg estiamte val_loss,
                                                       # patience is how much to wait for when loss is not improving

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(12, input_dim=8, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train,y_train, epochs=14, batch_size=10, validation_split = 0.1,callbacks=[stopping])
# evaluate the keras model
_, accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/14
1080/1080 [==============================] - 1s 726us/step - loss: 0.1751 - accuracy: 0.9430 - val_loss: 0.0828 - val_accuracy: 0.9717
Epoch 2/14
1080/1080 [==============================] - 1s 620us/step - loss: 0.0777 - accuracy: 0.9774 - val_loss: 0.0813 - val_accuracy: 0.9733
Epoch 3/14
1080/1080 [==============================] - 1s 610us/step - loss: 0.0747 - accuracy: 0.9784 - val_loss: 0.0795 - val_accuracy: 0.9733
Epoch 4/14
1080/1080 [==============================] - 1s 607us/step - loss: 0.0726 - accuracy: 0.9785 - val_loss: 0.0791 - val_accuracy: 0.9717
Epoch 5/14
1080/1080 [==============================] - 1s 611us/step - loss: 0.0713 - accuracy: 0.9792 - val_loss: 0.0784 - val_accuracy: 0.9725
Epoch 6/14
1080/1080 [==============================] - 1s 600us/step - loss: 0.0702 - accuracy: 0.9794 - val_loss: 0.0778 - val_accuracy: 0.9742
Epoch 7/14
1080/1080 [==============================] - 1s 590us/step - loss: 0.0697 - accuracy: 0.9795 - val_loss: 0.0773 -

## Hyperparameter Tuning.
- There are several ways to tune Hyperparameters of neural netowrk to get optimal values. 

- Here we will use grid search to find optimal values

In [20]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#The following is a function. we can input any hyperparemeter we want to test out eg dropout,neurons.optimzer, here we only 
# want to tune optimzer
def build_classifier(optimizer):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(12, input_dim=8, activation='relu'))
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
# compile the keras model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

classifier = KerasClassifier(build_fn=build_classifier) # wrapper for function
parameters = {'batch_size': [32],  # these are the hyperparameters we want to test out
              'epochs': [5],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10) # ran for 10
grid_search = grid_search.fit(X_train, y_train) # fit on grid search


best_parameters = grid_search.best_params_ # best ones
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

Epoch 1/5
338/338 [==============================] - 0s 569us/step - loss: 0.2206 - accuracy: 0.9403
Epoch 2/5
338/338 [==============================] - 0s 578us/step - loss: 0.0872 - accuracy: 0.9753
Epoch 3/5
338/338 [==============================] - 0s 594us/step - loss: 0.0792 - accuracy: 0.9767
Epoch 4/5
338/338 [==============================] - 0s 587us/step - loss: 0.0759 - accuracy: 0.9773
Epoch 5/5
338/338 [==============================] - 0s 590us/step - loss: 0.0737 - accuracy: 0.9782
Epoch 1/5
338/338 [==============================] - 0s 556us/step - loss: 0.2303 - accuracy: 0.9434
Epoch 2/5
338/338 [==============================] - 0s 577us/step - loss: 0.0827 - accuracy: 0.9765
Epoch 3/5
338/338 [==============================] - 0s 575us/step - loss: 0.0775 - accuracy: 0.9765
Epoch 4/5
338/338 [==============================] - 0s 576us/step - loss: 0.0751 - accuracy: 0.9773
Epoch 5/5
338/338 [==============================] - 0s 582us/step - loss: 0.0733 - accurac

- Downside of grid search is that it takes long time